# Librerias

In [ ]:
random_state = 43992294
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import category_encoders as ce
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.impute import SimpleImputer
import seaborn as sns
from scipy.stats import chi2_contingency
import scipy.stats as ss
import os
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MultiLabelBinarizer

from xgboost import XGBClassifier
from sklearn.preprocessing import FunctionTransformer

import ast

import warnings

from tqdm.auto import tqdm

from sklearn.feature_selection import RFECV

from sklearn.model_selection import StratifiedKFold, cross_val_score

from itertools import chain

import dask.dataframe as dd

# Cython imports
from tools import agrupar_categorias_cython, custom_one_hot_encoder_cython

# Fuciones

In [2]:
# Desactivar warnings
warnings.filterwarnings('ignore')

def augment_train_data(main_train_df, supplementary_df, umbral_raras=100):
    """
    Agrega filas del dataset suplementario al conjunto de entrenamiento principal
    basándose en categorías desconocidas y raras, evitando la duplicación de filas.

    Parámetros:
    - main_train_df (pd.DataFrame): DataFrame principal de entrenamiento.
    - supplementary_df (pd.DataFrame): DataFrame suplementario del cual se extraerán las filas.
    - umbral_raras (int): Umbral de frecuencia para considerar una categoría como rara.

    Retorna:
    - main_train_df (pd.DataFrame): DataFrame de entrenamiento actualizado.
    - categorias_desconocidas (dict): Diccionario actualizado de categorías desconocidas.
    - categorias_raras (dict): Diccionario actualizado de categorías raras.
    """
    # Definir columnas que no deseas tratar como categóricas
    columns_to_exclude = ['auction_list_0', 'action_list_1', 'action_list_2', 'auction_time']

    # Identificar columnas categóricas excluyendo las especificadas
    categorical_features = main_train_df.select_dtypes(include=['object']).columns.tolist()
    categorical_features = [col for col in categorical_features if col not in columns_to_exclude]
    
    # Crear un diccionario para almacenar las categorías desconocidas por columna
    categorias_desconocidas = {}
    
    # Iterar a través de cada columna categórica para identificar categorías desconocidas
    for columna in categorical_features:
        # Obtener las categorías únicas en el conjunto de entrenamiento
        categorias_train = set(main_train_df[columna].dropna().unique())
        
        # Obtener las categorías únicas en el dataset suplementario
        categorias_suplementario = set(supplementary_df[columna].dropna().unique())
        
        # Identificar las categorías en el dataset suplementario que no están en el entrenamiento
        desconocidas = categorias_suplementario - categorias_train
        
        # Almacenar las categorías desconocidas en el diccionario como una lista
        categorias_desconocidas[columna] = list(desconocidas)
    
    # Inicializar el diccionario para almacenar las categorías raras por columna
    categorias_raras = {}
    
    # Identificar categorías raras en el conjunto de entrenamiento
    for columna in categorical_features:
        # Contar la frecuencia de cada categoría
        frecuencia = main_train_df[columna].value_counts()
        
        # Identificar categorías que aparecen menos de umbral_raras veces
        raras = frecuencia[frecuencia < umbral_raras].index.tolist()
        
        # Almacenar en el diccionario
        categorias_raras[columna] = raras
    
    # Crear una máscara booleana para filas con categorías desconocidas o raras
    mask_desconocidas = pd.Series([False] * len(supplementary_df))
    mask_raras = pd.Series([False] * len(supplementary_df))
    
    for columna in categorical_features:
        # Actualizar la máscara para categorías desconocidas
        if categorias_desconocidas[columna]:
            mask_desconocidas = mask_desconocidas | supplementary_df[columna].isin(categorias_desconocidas[columna])
        
        # Actualizar la máscara para categorías raras
        if categorias_raras[columna]:
            mask_raras = mask_raras | supplementary_df[columna].isin(categorias_raras[columna])
    
    # Combinar ambas máscaras
    mask_total = mask_desconocidas | mask_raras
    
    # Filtrar filas únicas a agregar
    filas_a_agregar = supplementary_df[mask_total].drop_duplicates()
    
    # Mostrar información de agregación
    total_agregadas = len(filas_a_agregar)
    print(f"\nAgregando {total_agregadas} filas del dataset suplementario basadas en categorías desconocidas o raras.")
    
    # Agregar las filas al conjunto de entrenamiento
    main_train_df = pd.concat([main_train_df, filas_a_agregar], ignore_index=True)
    
    # Actualizar los diccionarios eliminando las categorías que ya han sido agregadas
    for columna in categorical_features:
        # Actualizar categorías desconocidas
        if categorias_desconocidas[columna]:
            categorias_agregadas = filas_a_agregar[columna].unique().tolist()
            categorias_desconocidas[columna] = [cat for cat in categorias_desconocidas[columna] if cat not in categorias_agregadas]
        
        # Actualizar categorías raras
        if categorias_raras[columna]:
            # Recontar la frecuencia después de agregar
            frecuencia = main_train_df[columna].value_counts()
            nuevas_raras = frecuencia[frecuencia < umbral_raras].index.tolist()
            categorias_raras[columna] = nuevas_raras
    
    return main_train_df, categorias_desconocidas, categorias_raras

def expand_list_dummies_cython(df, column, delimiter='|', prefix=None, suffix=None):
    """
    Expande una columna que contiene listas en múltiples columnas binarias usando un one-hot encoder optimizado con Cython.
    
    Parameters:
    - df (pd.DataFrame): DataFrame de pandas.
    - column (str): Nombre de la columna a expandir.
    - delimiter (str): Delimitador a usar en get_dummies (por defecto '|').
    - prefix (str, optional): Prefijo para las nuevas columnas binarias.
    - suffix (str, optional): Sufijo para las nuevas columnas binarias.
    
    Returns:
    - df_expanded (pd.DataFrame): DataFrame con las nuevas columnas binarias añadidas y la columna original eliminada.
    """
    # Copiar el DataFrame para evitar modificar el original
    df = df.copy()
    
    # Nombre para el valor desconocido
    unknown_value = f'Desconocido_{column[len(column)-1]}'
    
    # Reemplazar NaN por el valor desconocido
    df[column] = df[column].fillna(unknown_value)
    
    # Convertir las cadenas que representan listas en listas reales de Python
    def parse_list(x):
        try:
            parsed = ast.literal_eval(x)
            if isinstance(parsed, list) and len(parsed) == 0:
                # Tratar listas vacías como 'Desconocido'
                return [unknown_value]
            elif isinstance(parsed, list):
                # Convertir números a strings
                return [str(item) for item in parsed]
            else:
                return [str(x)]
        except (ValueError, SyntaxError):
            return [unknown_value]
    
    df[column] = df[column].apply(parse_list)
    
    # Convertir la columna en una lista de listas para pasarla a la función Cythonizada
    data_list = df[column].tolist()
    
    # Llamar a la función optimizada en Cython
    unique_categories, binary_matrix = custom_one_hot_encoder_cython(data_list)
    
    # Crear un DataFrame binario usando la matriz devuelta por Cython
    binary_df = pd.DataFrame(binary_matrix, index=df.index, columns=unique_categories)
    
    # Añadir prefijo y/o sufijo si se especifica
    if prefix:
        binary_df = binary_df.add_prefix(f"{prefix}_")
    if suffix:
        binary_df = binary_df.add_suffix(f"_{suffix}")
    
    # Concatenar las columnas binarias al DataFrame original, asegurando que solo se asigna 1, no se suma
    for col in binary_df.columns:
        if col in df.columns:
            df[col] = np.where((df[col] == 1) | (binary_df[col] == 1), 1, 0)
        else:
            df[col] = binary_df[col]
    
    # Eliminar la columna original ya que ha sido expandida
    df = df.drop(columns=[column])
    
    return df

def categorizar_hora(hora):
    """
    Categoriza una hora dada en 'mañana', 'tarde' o 'noche'.

    Parameters:
    - hora: Cadena de tiempo en formato 'HH:MM:SS' o un valor nulo.

    Returns:
    - Categoría de la parte del día: 'mañana', 'tarde' o 'noche'.
    """
    # Verificar si la hora es nula o no es una cadena
    if pd.isna(hora) or not isinstance(hora, str):
        return 'h_desconocido'
    
    # Extraer la hora como entero
    try:
        hora_int = int(hora.split(':')[0])
        if 0 <= hora_int < 12:
            return 'mañana'
        elif 12 <= hora_int < 18:
            return 'tarde'
        else:
            return 'noche'
    except (ValueError, IndexError):
        return 'h_desconocido'

def convertir_auction_time(df):
    """
    Convierte la columna 'auction_time' de formato timestamp a categorías de parte del día ('mañana', 'tarde', 'noche')
    y crea columnas binarias para cada categoría.

    Parameters:
    - df: DataFrame de pandas.

    Returns:
    - Lista de categorías creadas.
    - df: DataFrame con las nuevas columnas binarias añadidas y las columnas originales eliminadas.
    """
    # Convertir 'auction_time' de timestamp a cadena de tiempo 'HH:MM:SS', manejar NaN
    df['auction_time'] = pd.to_datetime(df['auction_time'], unit='s', errors='coerce').dt.strftime('%H:%M:%S')
    
    # Crear una nueva columna con las categorías de parte del día
    df['parte_del_dia'] = df['auction_time'].apply(categorizar_hora)
    
    # Crear columnas binarias para cada parte del día
    df['mañana'] = (df['parte_del_dia'] == 'mañana').astype(int)
    df['tarde'] = (df['parte_del_dia'] == 'tarde').astype(int)
    df['noche'] = (df['parte_del_dia'] == 'noche').astype(int)
    df['h_desconocido'] = (df['parte_del_dia'] == 'h_desconocido').astype(int)
    
    # Eliminar las columnas intermedias si no se necesitan
    df.drop(['parte_del_dia', 'auction_time'], axis=1, inplace=True)
    
    return df

def process_optimized(df):
    """
    Aplica una serie de transformaciones al DataFrame utilizando una función Cython optimizada.
    
    Parámetros:
    - df (pd.DataFrame): DataFrame a procesar.
    
    Retorna:
    - df (pd.DataFrame): DataFrame procesado.
    - categories (set): Conjunto de categorías creadas durante el procesamiento.
    """
    # Definir el número total de pasos para la barra de progreso
    total_steps = 5
    
    # Inicializar la barra de progreso
    with tqdm(total=total_steps, desc="Procesando DataFrame", unit="paso") as pbar:
        
        # 1. Agrupar categorías raras en 'Otro' y reemplazar NaN por 'Desconocidos' usando Cython
        # Preparar los datos
        columns_to_exclude = ['auction_list_0', 'action_list_1', 'action_list_2', 'auction_time']
        categorical_features = df.select_dtypes(include=['object']).columns.tolist()
        categorical_features = [col for col in categorical_features if col not in columns_to_exclude]
        
        # Convertir a matriz bidimensional
        data_matrix = df[categorical_features].values.tolist()
        data_matrix_cython = [list(row) for row in data_matrix]
        
        # Llamar a la función Cythonizada
        df_cython_data = agrupar_categorias_cython(
            categorical_features=categorical_features,
            columns_to_exclude=columns_to_exclude,
            data=data_matrix_cython,
            umbral=0                             # Con un umbral de 0, se agrupan solo los NaN
        )
        
        # Reasignar los datos al DataFrame
        for i, col in enumerate(categorical_features):
            df[col] = [row[i] for row in df_cython_data]
        
        pbar.update(1)  # Actualizar la barra de progreso
        
        # 2. Expandir la columna 'auction_list_0' en variables binarias
        df = expand_list_dummies_cython(df, 'auction_list_0')
        pbar.update(1)
        
        # 3. Expandir la columna 'action_list_1' en variables binarias
        df = expand_list_dummies_cython(df, 'action_list_1')
        pbar.update(1)
        
        # 4. Expandir la columna 'action_list_2' en variables binarias
        df = expand_list_dummies_cython(df, 'action_list_2')
        pbar.update(1)
        
        # 5. Convertir 'auction_time' a categorías de parte del día y crear variables binarias
        df = convertir_auction_time(df)
        pbar.update(1)
    
    return df

def process_data_with_dask(df, npartitions=10):
    """
    Procesa un DataFrame utilizando Dask para distribuir el trabajo en varias particiones.
    Aplica la función process_optimized a cada partición del DataFrame.
    
    Parámetros:
    - df (pd.DataFrame): El DataFrame de pandas a procesar.
    - npartitions (int): Número de particiones en las que se dividirá el DataFrame para su procesamiento.
    
    Retorna:
    - final_df (pd.DataFrame): El DataFrame procesado y concatenado.
    """
    # Convertir el DataFrame de pandas a Dask con el número de particiones especificado
    dask_df = dd.from_pandas(df, npartitions=npartitions)
    
    # Aplicar la función process_optimized a cada partición
    dask_df = dask_df.map_partitions(lambda df_partition: process_optimized(df_partition))
    
    # Ejecutar el cálculo distribuido y convertir el resultado a pandas
    final_df = dask_df.compute()

    return final_df

# Limpieza de datos

## Cancatenamos datasets

In [ ]:
train_data_21 = pd.read_csv('data/ctr_21.csv')

In [13]:
train_data_20 = pd.read_csv('data/ctr_20.csv')
train_data_19 = pd.read_csv('data/ctr_19.csv')
train_data_18 = pd.read_csv('data/ctr_18.csv')
train_data_17 = pd.read_csv('data/ctr_17.csv')
train_data_16 = pd.read_csv('data/ctr_16.csv')
train_data_15 = pd.read_csv('data/ctr_15.csv')

In [ ]:
supplementary_datasets = [
    ('1', train_data_20),
    ('2', train_data_19),
    ('3', train_data_18),
    ('4', train_data_17),
    ('5', train_data_16),
    ('6', train_data_15)
]

train_data = train_data_21

In [ ]:
for nombre, dataset in supplementary_datasets:
    print(f"\nProcesando dataset {nombre}/{len(supplementary_datasets)}")
    train_data, categorias_desconocidas, categorias_raras = augment_train_data(train_data, dataset)

In [ ]:
print(train_data.shape[0])

print(train_data_20.shape[0] + train_data_19.shape[0] + train_data_18.shape[0] + train_data_17.shape[0] + train_data_16.shape[0] + train_data_15.shape[0] + train_data_21.shape[0])

In [ ]:
train_data = agrupar_categorias(train_data)

In [ ]:
train_data.to_csv('train_data_combined.csv', index=False)

## Una vez ya concatenado...

In [ ]:
train_data = pd.read_csv('train_data_combined.csv')

In [37]:
test = pd.read_csv('data/ctr_test.csv')

### Test vs Train

#### En test pero no en train

In [ ]:
columns_to_exclude = ['auction_list_0', 'action_list_1', 'action_list_2', 'auction_time']
categorical_features = train_data.select_dtypes(include=['object']).columns.tolist()
categorical_features = [col for col in categorical_features if col not in columns_to_exclude]

# Crear un diccionario para almacenar las categorías desconocidas por columna
categorias_desconocidas = {}

total = 0

# Iterar a través de cada columna categórica
for columna in categorical_features:
    # Obtener las categorías únicas en el conjunto de entrenamiento
    categorias_train = set(train_data[columna].dropna().unique())
    
    # Obtener las categorías únicas en el conjunto de prueba
    categorias_test = set(test[columna].dropna().unique())
    
    # Identificar las categorías en test que no están en train
    desconocidas = categorias_test - categorias_train
    
    # Almacenar las categorías desconocidas en el diccionario como una lista
    categorias_desconocidas[columna] = list(desconocidas)
    if len(desconocidas) > 0:
        print(f"'{columna}' ({len(desconocidas)}): {desconocidas} ")

In [ ]:
# Inicializamos una máscara booleana que será True si la fila tiene una categoría desconocida en cualquier columna
mask = pd.Series([False] * len(test))

# Iteramos a través de cada columna categórica
for columna in categorias_desconocidas:
    # Verificamos si el valor en la columna está dentro de las categorías desconocidas
    mask = mask | test[columna].isin(categorias_desconocidas[columna])

# Contamos el número de filas donde al menos una categoría es desconocida
num_filas_desconocidas = mask.sum()

print(f"Cantidad de filas con al menos una categoría desconocida: {num_filas_desconocidas} ({num_filas_desconocidas / len(test) * 100:.2f}%)")

#### En train pero no en test

In [ ]:
columns_to_exclude = ['auction_list_0', 'action_list_1', 'action_list_2', 'auction_time']
categorical_features = test.select_dtypes(include=['object']).columns.tolist()
categorical_features = [col for col in categorical_features if col not in columns_to_exclude]

# Crear un diccionario para almacenar las categorías desconocidas por columna
categorias_conocidas = {}

# Iterar a través de cada columna categórica
for columna in categorical_features:
    # Obtener las categorías únicas en el conjunto de entrenamiento
    categorias_test = set(test[columna].dropna().unique())
    
    # Obtener las categorías únicas en el conjunto de prueba
    categorias_train = set(train_data[columna].dropna().unique())
    
    # Identificar las categorías en test que no están en train
    conocidas = categorias_train - categorias_test
    
    # Almacenar las categorías desconocidas en el diccionario como una lista
    categorias_conocidas[columna] = list(conocidas)

In [ ]:
# Inicializamos una máscara booleana que será True si la fila tiene una categoría desconocida en cualquier columna
mask = pd.Series([False] * len(test))

# Iteramos a través de cada columna categórica
for columna in categorias_conocidas:
    # Verificamos si el valor en la columna está dentro de las categorías desconocidas
    mask = mask | train_data[columna].isin(categorias_conocidas[columna])

# Contamos el número de filas donde al menos una categoría es desconocida
num_filas_conocidas = mask.sum()

print(f"Cantidad de filas con al menos una categoría conocida: {num_filas_conocidas} ({num_filas_conocidas / len(train_data) * 100:.2f}%)")

In [ ]:
for columna in categorias_desconocidas:
    test[columna] = test[columna].apply(lambda x: 'Otro' if x in categorias_desconocidas[columna] else x)

for columna in categorias_conocidas:
    train_data[columna] = train_data[columna].apply(lambda x: 'Otro' if x in categorias_conocidas[columna] else x)

## Procesamiento de datos

In [ ]:
train_data_cleaned = process_data_with_dask(train_data, npartitions=20)

## Columas IDs y sus apariciones

In [18]:
# Columnas que son IDs
id_columns = [
    'action_categorical_0', 'action_categorical_1', 'action_categorical_2',
    'action_categorical_3', 'action_categorical_4',
    'action_list_1', 'action_list_2',
    'auction_categorical_0', 'auction_categorical_1', 'auction_categorical_7',
    'auction_categorical_8', 'auction_categorical_9', 'auction_categorical_11',
    'creative_categorical_0', 'creative_categorical_5',
    'device_id'
]

list_id_columns = ['action_list_1', 'action_list_2']

In [ ]:
for col in id_columns:
    # Excluir listas de IDs por ahora
    if col not in ['action_list_1', 'action_list_2']:
        total_ids = train_data[col].nunique()
        total_rows = train_data[col].shape[0]
        print(f"Columna '{col}': {total_ids} IDs únicos en {total_rows} filas.")
        
        if total_ids == total_rows:
            print(f"  -> Todos los IDs en '{col}' son únicos.\n")
        else:
            duplicados = train_data[col].duplicated().sum()
            print(f"  -> Hay {duplicados} IDs duplicados en '{col}'.\n")

for col in list_id_columns:
    # Expandir las listas de IDs en una sola lista
    all_ids = train_data[col].dropna().apply(lambda x: x.split(','))  # Asumiendo que las listas están separadas por comas
    all_ids = list(chain.from_iterable(all_ids))
    unique_ids = set(all_ids)
    total_ids = len(unique_ids)
    print(f"Columna '{col}': {total_ids} IDs únicos en todas las filas.")
    
    # Verificar si cada ID es único (no se repite en diferentes filas)
    counts = pd.Series(all_ids).value_counts()
    duplicados = counts[counts > 1].count()
    print(f"  -> Hay {duplicados} IDs que se repiten en múltiples filas.\n")

## Entrenar

### Target Encoding

In [ ]:
# Dividir los datos en entrenamiento y validación con estratificación
X_train, X_val, y_train, y_val = train_test_split(
    train_data_cleaned.drop(columns='Label'),  # Características
    train_data_cleaned['Label'],               # Variable objetivo
    test_size=0.2,                              # 20% para validación
    stratify=train_data_cleaned['Label'],      # Estratificación basada en la variable objetivo
    random_state=random_state                   # Semilla para reproducibilidad
)

# Verificar la proporción de clases en el conjunto de entrenamiento y validación
print("Proporción en el conjunto de entrenamiento:")
print(y_train.value_counts(normalize=True))

print("Proporción en el conjunto de validación:")
print(y_val.value_counts(normalize=True))

In [ ]:
# Definir las columnas categóricas a codificar
categorical_features_to_encode = col in X_train.select_dtypes(include=['object']).columns

# Crear el Target Encoder para las columnas categóricas
target_encoder = ce.TargetEncoder(cols=categorical_features_to_encode)

# Crear el imputador
imputer = SimpleImputer(strategy='mean')

model_xgb_te = XGBClassifier(random_state=random_state)

pipeline_xgb_te = Pipeline(steps=[
    ('target_encoder', target_encoder),  # Paso de Target Encoding
    ('imputer', imputer),                # Paso de imputación
    ('classifier', model_xgb_te)         # Paso de Xgboost
])